## Part 9 - Random Forest

In [102]:
import pip 
pip.main(["install","matplotlib"]) 

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
You should consider upgrading via the 'c:\Python39\python.exe -m pip install --upgrade pip' command.


0

In [103]:
!pip install --upgrade scikit-learn

Requirement already up-to-date: scikit-learn in c:\python39\lib\site-packages (1.1.2)


You should consider upgrading via the 'c:\python39\python.exe -m pip install --upgrade pip' command.


In [104]:
#from xgboost import XGBClassifier
#from sklearn.model_selection import GridSearchCV

In [105]:
# Chargement des librairies nécessaires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, RocCurveDisplay, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, roc_curve
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings("ignore", category=FutureWarning) # to avoid deprecation warnings

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

In [106]:
# Chargement de la dataset
## Choix de 4 datasets

#dataset = pd.read_csv("dataset_CV_labelise_features1_100.csv", delimiter = ";", encoding = "utf-8") ## 100CV + 4 features numériques

#dataset = pd.read_csv("dataset_CV_labelise_features1_200.csv", delimiter = ";", encoding = "utf-8") ## 200CV + 4 features numériques

#dataset = pd.read_csv("dataset_CV_labelise_features2_100.csv", delimiter = ";", encoding = "utf-8") ## 100CV + 12 features numériques

dataset = pd.read_csv("dataset_CV_labelise_features2_200.csv", delimiter = ";", encoding = "utf-8") ## 200CV + 12 features numériques

In [107]:
dataset.shape

(5594, 19)

In [108]:
data_desc = dataset.describe(include='all')
display(data_desc)

,CV_Sentences,Sentences_CV_clean,CV_Number,Sentence_line,Nb_tokens,%texte_lu,%texte_lu_fin_ligne,Is_alpha,Grammar,Tokenization,Verb_count,Propn_count,Noun_count,Num_count,Pourcentage_verb_sentence,Pourcentage_propn_sentence,Pourcentage_noun_sentence,Pourcentage_num_sentence,Label
count,5594,5592,5594,5594.000000,5594.000000,5594.000000,5594.000000,5594,5594,5592,5594.000000,5594.000000,5594.00000,5594.000000,5594.000000,5594.000000,5594.000000,5594.000000,5594.000000
unique,3805,3802,200,NaN,NaN,NaN,NaN,1626,2748,3802,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,,,CV_10,NaN,NaN,NaN,NaN,[False],['NOUN'],,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,198,198,66,NaN,NaN,NaN,NaN,638,334,198,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,15.673221,9.132285,50.470518,49.529483,NaN,NaN,NaN,0.297283,0.611012,2.83822,0.320164,0.020117,0.105039,0.314979,0.042982,0.037898
std,NaN,NaN,NaN,11.268076,10.914849,30.190027,30.190025,NaN,NaN,NaN,0.784780,1.129198,3.56600,0.693458,0.059959,0.227969,0.251108,0.131906,0.190966
min,NaN,NaN,NaN,0.000000,1.000000,0.240000,0.000000,NaN,NaN,NaN,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,NaN,NaN,NaN,7.000000,2.000000,24.190000,23.552500,NaN,NaN,NaN,0.000000,0.000000,1.00000,0.000000,0.000000,0.000000,0.157895,0.000000,0.000000
50%,NaN,NaN,NaN,14.000000,5.000000,49.530000,50.470000,NaN,NaN,NaN,0.000000,0.000000,2.00000,0.000000,0.000000,0.000000,0.307692,0.000000,0.000000
75%,NaN,NaN,NaN,23.000000,12.000000,76.447500,75.810000,NaN,NaN,NaN,0.000000,1.000000,4.00000,0.000000,0.000000,0.102564,0.416667,0.000000,0.000000


In [109]:
display(100*dataset.isnull().sum()/dataset.shape[0])

CV_Sentences                  0.000000
Sentences_CV_clean            0.035753
CV_Number                     0.000000
Sentence_line                 0.000000
Nb_tokens                     0.000000
%texte_lu                     0.000000
%texte_lu_fin_ligne           0.000000
Is_alpha                      0.000000
Grammar                       0.000000
Tokenization                  0.035753
Verb_count                    0.000000
Propn_count                   0.000000
Noun_count                    0.000000
Num_count                     0.000000
Pourcentage_verb_sentence     0.000000
Pourcentage_propn_sentence    0.000000
Pourcentage_noun_sentence     0.000000
Pourcentage_num_sentence      0.000000
Label                         0.000000
dtype: float64

In [110]:
dataset["Label"].unique()
dataset = dataset.loc[(dataset['CV_Sentences'] != "#NOM?"),:]
dataset.shape 

(5416, 19)

In [111]:
dataset = dataset.loc[(dataset['CV_Sentences'] != ":"),:]
dataset.shape

(5412, 19)

In [112]:
dataset = dataset.dropna(axis =0, how = 'any')
dataset.shape

(5410, 19)

In [113]:
display(100*dataset.isnull().sum()/dataset.shape[0])

CV_Sentences                  0.0
Sentences_CV_clean            0.0
CV_Number                     0.0
Sentence_line                 0.0
Nb_tokens                     0.0
%texte_lu                     0.0
%texte_lu_fin_ligne           0.0
Is_alpha                      0.0
Grammar                       0.0
Tokenization                  0.0
Verb_count                    0.0
Propn_count                   0.0
Noun_count                    0.0
Num_count                     0.0
Pourcentage_verb_sentence     0.0
Pourcentage_propn_sentence    0.0
Pourcentage_noun_sentence     0.0
Pourcentage_num_sentence      0.0
Label                         0.0
dtype: float64

In [114]:
print("Separating labels from features...")

## Choisir la features_list par rapport au dataset
#features_list = ["CV_Sentences","Sentences_CV_clean","CV_Number", "Is_alpha", "Grammar", "Label"] ## Pour les datasets features1
features_list = ["CV_Sentences","Sentences_CV_clean","CV_Number", "Is_alpha", "Label", "Grammar", "Tokenization"] ## Pour les datasets features2
target_variable = "Label"

X = dataset.drop(features_list, axis = 1)
y = dataset.loc[:,target_variable]

print('y : ')
print(y.head())
print()
print('X :')
print(X.head())

Separating labels from features...
y : 
0    1
1    0
2    0
3    0
4    0
Name: Label, dtype: int64

X :
   Sentence_line  Nb_tokens  %texte_lu  %texte_lu_fin_ligne  Verb_count  \
0              0          6       2.80                97.20           1   
1              1          1       3.27                96.73           0   
2              2          8       7.01                92.99           0   
3              3         24      18.22                81.78           5   
4              4          1      18.69                81.31           0   

   Propn_count  Noun_count  Num_count  Pourcentage_verb_sentence  \
0            3           1          1                   0.166667   
1            0           1          0                   0.000000   
2            1           2          0                   0.000000   
3            1           5          1                   0.208333   
4            0           1          0                   0.000000   

   Pourcentage_propn_sentence  Pou

In [115]:
# Automatically detect names of numeric/categorical columns
numeric_features = []
categorical_features = []
for i,t in X.dtypes.iteritems():
    if ('float' in str(t)) or ('int' in str(t)) :
        numeric_features.append(i)
    else :
        categorical_features.append(i)

print('Found numeric features ', numeric_features)
print('Found categorical features ', categorical_features)

Found numeric features  ['Sentence_line', 'Nb_tokens', '%texte_lu', '%texte_lu_fin_ligne', 'Verb_count', 'Propn_count', 'Noun_count', 'Num_count', 'Pourcentage_verb_sentence', 'Pourcentage_propn_sentence', 'Pourcentage_noun_sentence', 'Pourcentage_num_sentence']
Found categorical features  []


In [116]:
# Divide dataset Train set & Test set 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify = y)

In [117]:
# Create pipeline for numeric features
numeric_transformer = SimpleImputer(strategy='mean')

In [118]:
# Create pipeline for categorical features
categorical_transformer = OneHotEncoder(drop='first')

In [119]:
# Use ColumnTransformer to make a preprocessor object that describes all the treatments to be done
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [120]:
# Perform grid search
print("Grid search...")
classifier = RandomForestClassifier()

# Grid of values to be tested
params = {
    'max_depth': [2, 4, 6, 8, 10],
    'min_samples_leaf': [1, 2, 5],
    'min_samples_split': [2, 4, 8],
    'n_estimators': [10, 20, 40, 60, 80, 100]
}
gridsearch = GridSearchCV(classifier, param_grid = params, cv = 3) # cv : the number of folds to be used for CV
gridsearch.fit(X_train, y_train)
print("...Done.")
print("Best hyperparameters : ", gridsearch.best_params_)
print("Best validation accuracy : ", gridsearch.best_score_)

Grid search...
...Done.
Best hyperparameters :  {'max_depth': 8, 'min_samples_leaf': 2, 'min_samples_split': 8, 'n_estimators': 40}
Best validation accuracy :  0.9738913670952506


In [121]:
# Predictions on training set
print("Predictions on training set...")
Y_train_pred = gridsearch.predict(X_train)
print("...Done.")
print(Y_train_pred)
print()

Predictions on training set...
...Done.
[0 0 0 ... 0 0 0]



In [122]:
# Predictions on test set
print("Predictions on test set...")
Y_test_pred = gridsearch.predict(X_test)
print("...Done.")
print(Y_test_pred)
print()

Predictions on test set...
...Done.
[0 0 0 ... 0 0 0]



In [123]:
# Print scores
print("accuracy on training set : ", accuracy_score(y_train, Y_train_pred))
print("accuracy on test set : ", accuracy_score(y_test, Y_test_pred))
print()

print("f1-score on training set : ", f1_score(y_train, Y_train_pred))
print("f1-score on test set : ", f1_score(y_test, Y_test_pred))                              
print()

accuracy on training set :  0.9785120147874307
accuracy on test set :  0.9685767097966729

f1-score on training set :  0.6618181818181819
f1-score on test set :  0.46875000000000006



In [124]:
# Visualize confusion matrices
from plotly.subplots import make_subplots
cm_train = confusion_matrix(y_train, Y_train_pred)
cm_test = confusion_matrix(y_test, Y_test_pred)

fig = make_subplots(rows = 1, cols = 2, subplot_titles = ("train", "test"), 
                    x_title = 'Prediction', y_title = 'True label')
fig.update_layout(
        title = go.layout.Title(text = "Confusion matrices", x = 0.5))
fig.update_yaxes(autorange='reversed')
fig.add_trace(
    go.Heatmap(
        name = 'train',
        x = ['0', '1'], 
        y = ['0', '1'], 
        z = cm_train,
        colorscale = 'gnbu',
        zmin = 0,
        zmax = max(cm_train.max(), cm_test.max())
    ),
    row = 1,
    col = 1
)  
fig.add_trace(
    go.Heatmap(
        name = 'test',
        x = ['0', '1'], 
        y = ['0', '1'], 
        z = cm_test,
        colorscale = 'gnbu',
        zmin = 0,
        zmax = max(cm_train.max(), cm_test.max())
    ),
    row = 1,
    col = 2
)

In [125]:
# Visualize ROC curves
probas_train = gridsearch.predict_proba(X_train)[:,1]
precisions, recalls, thresholds = roc_curve(y_train, probas_train)
fig = go.Figure(
    data = go.Scatter(
        name = 'train',
        x = recalls, 
        y = precisions, 
        mode = 'lines'
    ),
    layout = go.Layout(
        title = go.layout.Title(text = "ROC curve", x = 0.5),
        xaxis = go.layout.XAxis(title = 'False Positive Rate'),
        yaxis = go.layout.YAxis(title = 'True Positive Rate')
    )
)

probas_test = gridsearch.predict_proba(X_test)[:,1]
precisions, recalls, thresholds = roc_curve(y_test, probas_test)
fig.add_trace(go.Scatter(
    name = 'test',
    x = recalls, 
    y = precisions, 
    mode = 'lines'
    )
)
fig.show()